In [1]:
import pandas as pd
df = pd.read_csv('processed_lyrics.csv')


In [2]:
subset = df.sample(n=1000)

In [3]:
subset

,ALink,SName,SLink,Lyric,Idiom,Genre,Artist
42402,/tarja-turunen/,500 Letters,/tarja-turunen/500-letters.html,500 letters from a stranger on my door. 500 le...,ENGLISH,Rock,Tarja Turunen
32037,/phoenix/,Love Like a Sunset Part I,/phoenix/love-like-a-sunset-part-i.html,Acres. Visible horizon. Right where it starts ...,ENGLISH,Rock,Phoenix
74592,/christopher-cross/,Never Be The Same,/christopher-cross/never-be-the-same.html,It was good for me. It was good for you. Now n...,ENGLISH,Pop,Christopher Cross
34745,/richard-thompson/,Honky Tonk Blues,/richard-thompson/honky-tonk-blues.html,Well I left my home down on the rural route. I...,ENGLISH,Rock,Richard Thompson/ Richard And Linda Thompson
12416,/elvis-costello/,Days,/elvis-costello/days.html,Thank you for the days. Those endless days tho...,ENGLISH,Rock,Elvis Costello
...,...,...,...,...,...,...,...
13908,/eric-clapton/,I'm So Glad,/eric-clapton/im-so-glad.html,"I'm so glad, I'm so glad. I'm glad, I'm glad, ...",ENGLISH,Rock,Eric Clapton
43708,/the-beatles/,From Fluff to You,/the-beatles/from-fluff-to-you.html,"Alan Freeman: Paul?. Paul: Yes?. Alan: Now, li...",ENGLISH,Rock,The Beatles
85928,/mika/,Let It Snow,/mika/let-it-snow.html,Oh the weather outside is frightful. But the f...,ENGLISH,Pop,Mika
80940,/jewel/,Styrofoam Dream,/jewel/styrofoam-dream.html,"Oh, buttercup. ...Lives in a cup.... She lost ...",ENGLISH,Pop,Jewel


In [13]:
from sklearn.model_selection import train_test_split
 
train, test = train_test_split(subset, test_size=0.2, random_state=42)
X_train = train.loc[:, train.columns != 'Genre']
y_train = train['Genre']
X_test = test.loc[:, test.columns != 'Genre']
y_test = test['Genre']

In [14]:
X_train

,ALink,SName,SLink,Lyric,Idiom,Artist
34362,/red-hot-chili-peppers/,In Love Dying,/red-hot-chili-peppers/in-love-dying.html,In love dying. There's a flawless rain. In lov...,ENGLISH,Red Hot Chili Peppers
64703,/soulja-boy/,Live Life,/soulja-boy/live-life.html,Man I feel good. My life consist of a lotta ha...,ENGLISH,Soulja Boy
63545,/rihanna/,Bitch Better Have My Money,/rihanna/bitch-better-have-my-money.html,"Yeah yo, Yeah yo. Moo la la, yeah yo. Bitch be...",ENGLISH,Rihanna
53079,/yes/,Owner Of A Lonely Heart,/yes/owner-of-a-lonely-heart.html,Move yourself. You always live your life. Neve...,ENGLISH,Yes
58111,/far-east-movement/,Eyes Never Lie,/far-east-movement/eyes-never-lie.html,"These eyes, never lie. Give it to me (Give it ...",ENGLISH,Far East Movement
...,...,...,...,...,...,...
32095,/pink-floyd/,Another Brick In The Wall,/pink-floyd/another-brick-in-the-wall.html,Parte1. Daddy's flown across the ocean. Leavin...,ENGLISH,Pink Floyd
42637,/teenage-fanclub/,Say No,/teenage-fanclub/say-no.html,Say no. I know you're gonna stay. With more th...,ENGLISH,Teenage Fanclub
27590,/nazareth/,Love Hurts,/nazareth/love-hurts.html,"Love hurts, Love scars, Love wounds' and marks...",ENGLISH,Nazareth
76481,/ellie-goulding/,Burn,/ellie-goulding/burn.html,"We, we don't have to worry about nothing. 'Cau...",ENGLISH,Ellie Goulding


In [9]:
import numpy as np

from sklearn.base import BaseEstimator, TransformerMixin

class PairedSentenceVectorizer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    # Vectorize a single sentence.
    def _transform1(self, sentence):
        sentence = sentence.split()
        vector = np.array(sum(nlp.vocab[word].vector for word in sentence))
        return (vector)

    # Vectorize a single row of the dataframe.
    def _transform2(self, row):
        # TODO: Replace the following line with your own code
        sentence = self._transform1(row.Lyric)
        return sentence

    def transform(self, X):
        return np.concatenate(
            [self._transform2(row).reshape(1, -1) for row in X.itertuples()]
        )

In [16]:
# TODO: Enter code here to implement the word embeddings classifier. Print the classification report.
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
import spacy
#!python -m spacy download en_core_web_lg
nlp = spacy.load('en_core_web_lg')
from sklearn.neural_network import MLPClassifier
pipe = Pipeline(steps=[('vectorizer', PairedSentenceVectorizer()),
                       ('clf', MLPClassifier(random_state=1,  max_iter=8, hidden_layer_sizes=(300, 300, 300)))])
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
target_names = subset['Genre'].unique()
print(classification_report(y_test, y_pred, target_names=target_names))

/Users/martinfriberg/opt/anaconda3/envs/NLP/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (8) reached and the optimization hasn't converged yet.
  warnings.warn(


              precision    recall  f1-score   support

        Rock       0.55      0.82      0.66        50
         Pop       0.31      0.30      0.31        50
     Hip Hop       0.68      0.53      0.60       100

    accuracy                           0.55       200
   macro avg       0.52      0.55      0.52       200
weighted avg       0.56      0.55      0.54       200

